<a href="https://colab.research.google.com/github/JALVARADORUIZ/TFM_UB/blob/main/TFM_Pre_procesado_de_Im%C3%A1genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **PREPARACIÓN DEL ENTORNO**


In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

In [ ]:
from keras.applications.vgg16 import VGG16 
from keras.models import Model

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle

#### **PREPROCESADO DE IMÁGENES**


In [ ]:
path = r"/content/drive/MyDrive/TFM_Máster BDDS/datawine/vintage-images"
os.chdir(path)


In [ ]:
wines = []

with os.scandir(path) as files:
    for file in files:
        wines.append(file.name)

In [ ]:

img = load_img(wines[0], target_size=(224,224))

img = np.array(img)

print(img.shape)

(224, 224, 3)


/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
reshaped_img = img.reshape(1,224,224,3)
print(reshaped_img.shape)


(1, 224, 224, 3)


In [ ]:
x = preprocess_input(reshaped_img)

In [ ]:
model = VGG16()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

553476096/553467096 [==============================] - 2s 0us/step


In [ ]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [ ]:
data = {}
p = r"/content/drive/MyDrive/TFM_Máster BDDS/datawine/wine_features.pkl"

# loop through each image in the dataset
for wine in wines:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(wine,model)
        data[wine] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)
          
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

In [ ]:
# get a list of just the features
feat = np.array(list(data.values()))
feat.shape

(7618, 1, 4096)

In [ ]:
# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)
feat.shape

(7618, 4096)

In [ ]:
df_labels = pd.read_csv('/content/drive/MyDrive/TFM_Máster BDDS/datawine/df_clean.csv')
#25/09/22; PVallejo: cargo archivo df_clean2 que incorpora variable color
#df_labels = pd.read_csv('/content/drive/MyDrive/TFM_Máster BDDS/datawine/df_clean2.csv')

In [ ]:
df_images = pd.DataFrame(feat)

In [ ]:
df_imagesf = pd.concat([df_filenames,df_images], axis=1)

,image_clean,0,1,2,3,4,5,6,7,8,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,XdgggMtUQ0iMhEMkprUw-g_pl_480x640.png,0.0,0.0,0.0,0.0,0.587556,0.625139,0.0,0.0,0.032004,...,0.0,0.17697,0.0,0.421629,0.0,0.0,0.0,0.0,1.485473,0.267109


In [ ]:
#25/09/22; PVallejo: Este código lo usé para genera archivo df_img_processed2 que se elabora a partir de df_clean2.csv
#df_filenames = pd.DataFrame(filenames, columns=['image_clean'])
#df_imagesf = pd.concat([df_filenames,df_images], axis=1)
#df_imagesf

In [ ]:
dff = df_labels.merge(df_imagesf, how='left', on ='image_clean')

In [ ]:
dff1 = dff.drop([513, 610, 2461, 2628, 2784, 2967, 3857, 4136, 5179, 5819, 5928, 6194, 6707, 6917, 7030, 7042, 7472],axis=0)

In [ ]:
#dff1.to_csv('/content/drive/MyDrive/TFM_Máster BDDS/datawine/df_img_processed2.csv', index=False) --> #25/09/22; PVallejo: con archivo df_clean2 que incorpora color.
dff1.to_csv('/content/drive/MyDrive/TFM_Máster BDDS/datawine/df_img_processed.csv', index=False)